# Docking

Analysis of flexible docking calculations with GNINA with the Vina scoring function.

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

## Data

RMSDs was computed for each system separately and have been collected into a single file:

In [2]:
rmsd_file = "../carlos_cd/rmsds2.csv"

In [3]:
df = pd.read_csv(rmsd_file)
df

,pocket,protein,ligand,rank,rmsd,obrmsd,flexrmsd,fmaxrmsd,score
0,AA2AR,3EML,3QAK,0,2.22358,2.22358,2.30279,4.99695,-9.13664
1,AA2AR,3EML,3QAK,10,3.25554,3.25554,2.45832,4.41412,-12.32127
2,AA2AR,3EML,3QAK,11,2.51549,2.51549,2.36026,3.85305,-12.21410
3,AA2AR,3EML,3QAK,12,6.50215,6.50215,2.35526,4.50800,-12.17950
4,AA2AR,3EML,3QAK,13,10.29577,10.29580,2.31750,5.05064,-12.05803
...,...,...,...,...,...,...,...,...,...
166385,XIAP,5C84,4KMP,5,9.60539,9.60539,1.61861,2.28624,-7.57331
166386,XIAP,5C84,4KMP,6,8.41238,8.41238,1.41767,2.26065,-7.51172
166387,XIAP,5C84,4KMP,7,6.88340,6.88340,1.24734,2.14065,-7.40088
166388,XIAP,5C84,4KMP,8,5.19961,5.19961,2.02988,2.79305,-7.32203


Check that the number of systems is the expected number (`7970`):

In [4]:
# Get tuples from [pocket, ligand, protein] and count unique tuples
# Check that the number of unique tuples equals the total number protein-ligand pairs
allsystems = pd.unique(df[["pocket", "ligand", "protein"]].apply(tuple,axis="columns"))
print(len(allsystems))
assert len(allsystems) == 7970

7970


Drop all rows with `NaN` values:

In [5]:
df_clean = df.dropna()
systems = pd.unique(df_clean[["pocket", "ligand", "protein"]].apply(tuple,axis="columns"))
len(systems)

7943

Some systems have `NaN` values. We expect the systems in `invalid.lst` to contain `NaN` values since there are no flexible residues. In order to see which systems contained `NaN` we can take the difference between the two sets of systems:

In [6]:
diff = set(allsystems) - set(systems)
diff

{('ACES', '1ACJ', '1JJB'),
 ('ACES', '1ZGB', '1JJB'),
 ('ACES', '2CMF', '1JJB'),
 ('CDK2', '3IG7', '3QQJ'),
 ('HXK4', '3S41', '4DCH'),
 ('HXK4', '4NO7', '4DCH'),
 ('IGF1R', '1JQH', '5FXR'),
 ('IGF1R', '2OJ9', '5FXR'),
 ('IGF1R', '2ZM3', '5FXR'),
 ('IGF1R', '3LVP', '5FXR'),
 ('IGF1R', '3NW6', '5FXR'),
 ('IGF1R', '3NW7', '5FXR'),
 ('JAK2', '4D0W', '4F08'),
 ('JAK2', '4E4M', '4F08'),
 ('JAK2', '5CF6', '4F08'),
 ('MET', '5HOA', '3CCN'),
 ('MK01', '4FV2', '4GSB'),
 ('MK01', '4ZZM', '4GSB'),
 ('MK01', '5LCJ', '4GSB'),
 ('MK01', '5NHV', '4GSB'),
 ('MK10', '2G01', '1UKI'),
 ('MK10', '3ELJ', '1UKI'),
 ('MK10', '3RTP', '1UKI'),
 ('MK10', '4L7F', '4HYS'),
 ('SRC', '3DQX', '3UQG'),
 ('SRC', '5D10', '3UQG'),
 ('SRC', '5J5S', '3UQG')}

In [7]:
len(diff)

27

In order to look at the systems that have RMSD-related problems, we can load the systems already identified in `invalid.lst`

In [8]:
invalid = []
with open("invalid.lst", "r") as f:
    for line in f:
        l = line.strip().split()
        # Invalid list contains systems as [pocket, protein, ligand]
        # Here we are using [pocket, ligand, protein]
        invalid.append((l[0], l[2], l[1]))
invalid = set(invalid)
assert len(invalid) == 24 # 24 systems without flexible residues
invalid

{('ACES', '1ACJ', '1JJB'),
 ('ACES', '1ZGB', '1JJB'),
 ('ACES', '2CMF', '1JJB'),
 ('CDK2', '3IG7', '3QQJ'),
 ('IGF1R', '1JQH', '5FXR'),
 ('IGF1R', '2OJ9', '5FXR'),
 ('IGF1R', '2ZM3', '5FXR'),
 ('IGF1R', '3LVP', '5FXR'),
 ('IGF1R', '3NW6', '5FXR'),
 ('IGF1R', '3NW7', '5FXR'),
 ('JAK2', '4D0W', '4F08'),
 ('JAK2', '4E4M', '4F08'),
 ('JAK2', '5CF6', '4F08'),
 ('MK01', '4FV2', '4GSB'),
 ('MK01', '4ZZM', '4GSB'),
 ('MK01', '5LCJ', '4GSB'),
 ('MK01', '5NHV', '4GSB'),
 ('MK10', '2G01', '1UKI'),
 ('MK10', '3ELJ', '1UKI'),
 ('MK10', '3RTP', '1UKI'),
 ('MK10', '4L7F', '4HYS'),
 ('SRC', '3DQX', '3UQG'),
 ('SRC', '5D10', '3UQG'),
 ('SRC', '5J5S', '3UQG')}

In [9]:
def show_system(t):
    return df[(df.pocket == t[0]) & (df.ligand == t[1]) & (df.protein == t[2])]

In [10]:
set(allsystems) - set(systems) - invalid

{('HXK4', '3S41', '4DCH'), ('HXK4', '4NO7', '4DCH'), ('MET', '5HOA', '3CCN')}

If we look at one of the systems that were in the `invalid.lst` we can see that the problem is that there are no flexible residues (for which the RMSD is `NaN`):

In [11]:
show_system(('CDK2', '3IG7', '3QQJ'))

,pocket,protein,ligand,rank,rmsd,obrmsd,flexrmsd,fmaxrmsd,score
29482,CDK2,3QQJ,3IG7,0,2.48215,2.48215,NaN,NaN,-8.0795


For some systems not in the original `invalid.lst` list, only the flexible residue RMSD computed with `spyrmsd` is `NaN`. This is due to `spyrmsd` being killed after a while for large systems:

In [12]:
# flexobrmsd is non-null while flexrmsd is NaN
# For some (large) systems, spyrmsd process is killed after a while...
# Use flexobrmsd only
show_system(('HXK4', '3S41', '4DCH'))

,pocket,protein,ligand,rank,rmsd,obrmsd,flexrmsd,fmaxrmsd,score
78236,HXK4,4DCH,3S41,0,1.40188,1.40188,NaN,NaN,-3.88321
78237,HXK4,4DCH,3S41,10,9.05919,9.05919,NaN,NaN,-8.54325
78238,HXK4,4DCH,3S41,11,8.11876,8.11876,NaN,NaN,-8.21540
78239,HXK4,4DCH,3S41,12,7.31299,7.31299,NaN,NaN,-8.18070
78240,HXK4,4DCH,3S41,13,7.91945,7.91945,NaN,NaN,-8.08570
78241,HXK4,4DCH,3S41,14,6.94460,6.94460,NaN,NaN,-8.02355
78242,HXK4,4DCH,3S41,15,8.87998,8.87998,NaN,NaN,-7.98356
78243,HXK4,4DCH,3S41,16,9.14961,9.14961,NaN,NaN,-7.91726
78244,HXK4,4DCH,3S41,17,9.74881,9.74881,NaN,NaN,-7.81538
78245,HXK4,4DCH,3S41,18,9.43684,9.43684,NaN,NaN,-7.78488


In [13]:
# Remove all infinity values (if any)
df_clean = df.replace([np.inf, -np.inf], np.nan)
df_clean.dropna(inplace=True)

Remove the following systems manually because identified as problematic (broken disulfide bonds, added spurious bonds, ...) when computing target-pose RMSD:

In [14]:
def sysdrop(df, tp):
    pocket, protein, ligand = tp
    return df[~((df.pocket == pocket) & (df.protein == protein) & (df.ligand == ligand))]

to_drop = [
    ('CP2C9','1R9O','5W0C'),
    ('FA10', '2XBV', '2FZZ'),
    ('FA10', '1IQE', '2RA0'),
    ('FA10', '2XBV', '2Y82'),
    ('FA10', '1IQE', '3KQB'),
    ('KIF11', '4BXN', '1X88'),
    ('KIF11', '4BXN', '2IEH'),
    ('KIF11', '4BXN', '2X7D'),
    ('KIF11', '4BXN', '3K3B')
]

for d in to_drop:
    print(d)
    df_clean = sysdrop(df_clean, d)

('CP2C9', '1R9O', '5W0C')
('FA10', '2XBV', '2FZZ')
('FA10', '1IQE', '2RA0')
('FA10', '2XBV', '2Y82')
('FA10', '1IQE', '3KQB')
('KIF11', '4BXN', '1X88')
('KIF11', '4BXN', '2IEH')
('KIF11', '4BXN', '2X7D')
('KIF11', '4BXN', '3K3B')


In [15]:
systems = pd.unique(df_clean[["pocket", "ligand", "protein"]].apply(tuple,axis="columns"))
len(systems)

7934

In [16]:
df_clean.to_csv("rmsd2_clean.csv", float_format="%.5f", index=False)